In [1]:
%matplotlib inline
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

import os
import cv2
import numpy as np
import torch
from torch.autograd import Variable
import quat_math
import pickle
import glob

from PIL import Image
import scipy.io as scio
from functools import partial
from object_pose_utils.utils import to_np, to_var
from object_pose_utils.utils.display import *

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 20, 12
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import warnings
warnings.filterwarnings('ignore')

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


## Select Object Indices of Interest

| Object Indices |[]()|[]()|
|---|---|---|
| __1.__ 002_master_chef_can | __8.__ 009_gelatin_box      | __15.__ 035_power_drill       |
| __2.__ 003_cracker_box     | __9.__ 010_potted_meat_can  | __16.__ 036_wood_block        |
| __3.__ 004_sugar_box       | __10.__ 011_banana          | __17.__ 037_scissors          |
| __4.__ 005_tomato_soup_can | __11.__ 019_pitcher_base    | __18.__ 040_large_marker      |
| __5.__ 006_mustard_bottle  | __12.__ 021_bleach_cleanser | __19.__ 051_large_clamp       |
| __6.__ 007_tuna_fish_can   | __13.__ 024_bowl            | __20.__ 052_extra_large_clamp |
| __7.__ 008_pudding_box     | __14.__ 025_mug             | __21.__ 061_foam_brick        |

In [2]:
from transforms3d.quaternions import quat2mat, mat2quat

def getPoseCNNQuat(data, obj):
    pose_idx = np.where(data['rois'][:,1].flatten()==obj)[0]
    if(len(pose_idx) == 0):
        return None
    else:
        pose_idx = pose_idx[0]
    pose = data['poses'][pose_idx]
    q = pose[:4][[1,2,3,0]]
    q /= np.linalg.norm(q)
    t = pose[4:7]
    return q

## LINEMOD Dataset

In [4]:
from object_pose_utils.datasets.linemod_dataset import LinemodDataset
from object_pose_utils.datasets.image_processing import ImageNormalizer
#from object_pose_utils.datasets.inplane_rotation_augmentation import InplaneRotator

from object_pose_utils.datasets.pose_dataset import OutputTypes as otypes

dataset_root = "/scratch/datasets/linemod/Linemod_preprocessed/"
object_list = [1,2,4,5,6,8,9,10,11,12,13,14,15]
mode = "test"
num_objects = 13 #number of object classes in the dataset
num_points = 500 #number of points on the input pointcloud

output_format = [otypes.OBJECT_LABEL,
                 otypes.QUATERNION, 
                 otypes.TRANSLATION, 
                 otypes.IMAGE_CROPPED,
                 otypes.DEPTH_POINTS_MASKED_AND_INDEXES,
                ]

dataset = LinemodDataset(dataset_root, mode=mode,                          
                         segnet_masks = True,
                         object_list = object_list,
                         output_data = output_format,
                         resample_on_error = False,
                         add_syn_background = False,
                         add_syn_noise = False,
                         use_posecnn_data = True,
                         #preprocessors = [InplaneRotator()],
                         postprocessors = [ImageNormalizer()],
                         image_size = [640, 480], num_points=500)


## Pose Estimator

In [15]:
df_weights = '/home/bokorn/src/DenseFusion/trained_checkpoints/linemod/pose_model_9_0.01310166542980859.pth'
df_refine_weights = '/home/bokorn/src/DenseFusion/trained_checkpoints/linemod/pose_refine_model_493_0.006761023565178073.pth'

In [22]:
from dense_fusion.network import PoseNet, PoseRefineNet
from generic_pose.utils.evaluation_utils import fullEvaluateDenseFusion

from tqdm import tqdm_notebook as tqdm
import pickle

if(df_refine_weights is not None):
    df_refine_estimator = PoseRefineNet(num_points = num_points, num_obj = num_objects)
    df_refine_estimator.load_state_dict(torch.load(df_refine_weights))
    df_refine_estimator.cuda();
    df_refine_estimator.eval();
else:
    df_refine_estimator = None

df_estimator = PoseNet(num_points = num_points, num_obj = num_objects)

df_estimator.load_state_dict(torch.load(df_weights))
df_estimator.cuda();
df_estimator.eval();

results = {}

with torch.no_grad():
    for j, data in enumerate(tqdm(dataset)):
        obj, quat, trans, img, points, choose = data

        if(len(obj) == 0):
            results[j] = None
            continue

        obj = torch.nonzero(torch.LongTensor(dataset.object_labels) == obj)[0] + 1
        
        res = fullEvaluateDenseFusion(df_estimator, img, points, choose, obj)
        max_q, max_t, max_c, max_feat, pred_q, pred_t, pred_c, refine_q, refine_t, global_feat = res
        
        results[j] = {'max_q':max_q,
                      'max_t':max_t,
                      'max_c':max_c,
                      'max_feat':max_feat,
                      'pred_q':pred_q,
                      'pred_t':pred_t,
                      'pred_c':pred_c,
                      'refine_q':refine_q,
                      'refine_t':refine_t,
                      'global_feat':global_feat}

results_filename = 'results_linemod_test_df_' + '.'.join(df_weights.split('/')[-1].split('.')[:-1])
if(df_refine_weights is not None):
    results_filename += '_' + '.'.join(df_refine_weights.split('/')[-1].split('.')[:-1])
results_filename += '.pkl'

with open(results_filename, 'wb') as f:
    pickle.dump(results, f, protocol=pickle.HIGHEST_PROTOCOL)

Exception on index 3099: Mask 0050/000588 has less than minimum number of pixels (49 < 50)
Exception on index 3100: Mask 0050/000605 has less than minimum number of pixels (0 < 50)
Exception on index 3101: Mask 0050/000610 has less than minimum number of pixels (0 < 50)
Exception on index 3102: Mask 0050/000620 has less than minimum number of pixels (0 < 50)
Exception on index 3103: Mask 0050/000623 has less than minimum number of pixels (0 < 50)
Exception on index 3104: Mask 0050/000629 has less than minimum number of pixels (0 < 50)
Exception on index 3105: Mask 0050/000633 has less than minimum number of pixels (0 < 50)
Exception on index 3106: Mask 0050/000640 has less than minimum number of pixels (0 < 50)
Exception on index 3107: Mask 0050/000642 has less than minimum number of pixels (0 < 50)
Exception on index 3108: Mask 0050/000644 has less than minimum number of pixels (0 < 50)
Exception on index 3109: Mask 0050/000651 has less than minimum number of pixels (0 < 50)
Exception

Exception on index 13714: Mask 0057/001705 has less than minimum number of pixels (11 < 50)
Exception on index 13715: Mask 0057/001708 has less than minimum number of pixels (0 < 50)
Exception on index 13716: Mask 0057/001709 has less than minimum number of pixels (0 < 50)
Exception on index 13717: Mask 0057/001712 has less than minimum number of pixels (0 < 50)
Exception on index 13724: Mask 0057/001740 has less than minimum number of pixels (36 < 50)
Exception on index 13728: Mask 0057/001747 has less than minimum number of pixels (0 < 50)
Exception on index 13729: Mask 0057/001748 has less than minimum number of pixels (0 < 50)
Exception on index 13732: Mask 0057/001774 has less than minimum number of pixels (24 < 50)



In [20]:
df_weights.split('/')[-2] == 'ycb_global'

True